In [118]:
import pandas as pd
import spacy
import en_core_web_sm
from scipy.spatial.distance import cosine
nlp = en_core_web_sm.load()

In [1]:
import pandas as pd 
import numpy as np
import sklearn
import random
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [2]:
data=pd.read_csv('Hotel_Reviews.csv', encoding='utf-8')
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [28]:
tag = data.Tags.str.extract(r'\[(.+)\]').iloc[:,0].str.split(',')

In [59]:
tag[10]

["' Leisure trip '",
 " ' Group '",
 " ' Duplex Double Room '",
 " ' Stayed 2 nights '",
 " ' Submitted from a mobile device '"]

In [44]:

tag1 = [list(map(lambda x:x[2:-2], tag[n])) for n in range(len(tag))]

In [50]:
df1=pd.DataFrame(tag1)


In [60]:
df1

,0,1,2,3,4,5
0,Leisure trip,Couple,Duplex Double Room,Stayed 6 nights,None,None
1,Leisure trip,Couple,Duplex Double Room,Stayed 4 nights,None,None
2,Leisure trip,Family with young children,Duplex Double Room,Stayed 3 nights,Submitted from a mobile device,None
3,Leisure trip,Solo traveler,Duplex Double Room,Stayed 3 nights,None,None
4,Leisure trip,Couple,Suite,Stayed 2 nights,Submitted from a mobile device,None
5,Leisure trip,Group,Duplex Double Room,Stayed 1 night,None,None
6,Leisure trip,Group,Duplex Twin Room,Stayed 5 nights,Submitted from a mobile device,None
7,Leisure trip,Couple,Duplex Double Room,Stayed 2 nights,Submitted from a mobile device,None
8,Leisure trip,Couple,Duplex Double Room,Stayed 1 night,Submitted from a mobile device,None
9,Leisure trip,Couple,Large King Room,Stayed 7 nights,None,None


In [54]:
df1.iloc[:,0].value_counts()

Leisure trip                  416672
Business trip                  82748
Couple                          5808
Solo traveler                   4291
Family with young children      2227
Group                           1600
With a pet                      1405
Family with older children       931
Travelers with friends            56
Name: 0, dtype: int64

In [55]:
df1.iloc[:,1].value_counts()

 Couple                                                    245788
 Solo traveler                                             104077
 Group                                                      63663
 Family with young children                                 58582
 Family with older children                                 25331
 Travelers with friends                                      2087
 Leisure trip                                                1106
 Double Room                                                 1102
 Standard Double Room                                         937
 Superior Double Room                                         849
 Deluxe Double Room                                           771
 Double or Twin Room                                          587
 Standard Double or Twin Room                                 470
 Single Room                                                  419
 Standard Twin Room                                           383
 Classic D

In [121]:
import random 
positive = list(data['Negative_Review'])
samp = random.sample(positive, 10000)


## Word Embedding ##

In [110]:

review_embedding = []
random.seed(1)
for review in samp:
    review_embedding.append(nlp(review).vector)


KeyboardInterrupt: 

In [ ]:
import random
review_embedding = []
random.seed(1)
samp = random.sample(positive, 10000)

In [ ]:
review=pd.DataFrame(review_embedding) #2000 review vector
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
principalComponents = pca.fit_transform(review)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
principalDf

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(principalDf.iloc[:,0],principalDf.iloc[:,1])

## TF-IDF ##

In [126]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer()

lem_reviews = []
for review in samp:
    stem_words = nltk.word_tokenize(review)
    stem_words =[lemmatizer.lemmatize(word) for word in stem_words]
    lem_reviews.append(' '.join(stem_words))
    
vectorizer = TfidfVectorizer(ngram_range=(2,4),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features = 300, stop_words=stopwords.words())
                             
X = vectorizer.fit_transform(lem_reviews)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.transpose()




In [127]:
tf_idf.head()

,air condition,air conditioner,air conditioning,another hotel,another room,around hotel,back hotel,bad experience,bad smell,bar area,...,work room,work well,working properly,would better,would expect,would good,would like,would liked,would nice,would recommend
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.522339,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
tf_idf.sum(axis=0).sort_values(ascending=False)

room small                    148.317549
small room                    105.559383
air conditioning               78.702944
room service                   75.597415
star hotel                     61.784392
could better                   51.870711
breakfast included             48.860625
mini bar                       44.876515
room bit                       43.885291
breakfast expensive            43.489655
tea coffee                     40.890013
little bit                     38.762526
would nice                     37.161407
value money                    35.818497
even though                    35.184203
front desk                     33.393915
nothing like                   33.256415
breakfast room                 33.150314
size room                      32.581661
room little                    31.980432
reception staff                30.509333
could hear                     30.086203
bit small                      28.797451
room could                     28.795328
hot water       